In [3]:
import os
import requests
import csv
import pandas as pd
from google.cloud import bigquery

# Set Google Cloud Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"F:\project\pokemons-455012-175f0c65518c.json"

# Google Cloud configurations
PROJECT_ID = "pokemons-455012"  # Replace with your GCP project ID
DATASET_ID = "Pokemon_Master"
TABLE_ID = "Master_Data"

CSV_FILENAME = "pokemon_data.csv"
PROGRESS_FILE = "progress.txt"

# Initialize BigQuery Client
try:
    client = bigquery.Client()
    print("✅ Authentication successful!")
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    exit()

# Get the last fetched Pokémon index
if os.path.exists(PROGRESS_FILE):
    with open(PROGRESS_FILE, "r") as f:
        start_index = int(f.read().strip())
else:
    start_index = 1  # Start from the first Pokémon

# Fetch next 10 Pokémon
end_index = start_index + 9

# Check if CSV file exists
file_exists = os.path.exists(CSV_FILENAME)

# Open CSV file in append mode
with open(CSV_FILENAME, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write headers if the file doesn't exist
    if not file_exists:
        writer.writerow(["ID", "Name", "Type(s)", "Height", "Weight", "Base XP", "Abilities"])

    # Fetch Pokémon data from PokeAPI
    for pokemon_id in range(start_index, end_index + 1):
        url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            # Extract relevant details
            name = data["name"].capitalize()
            types = ", ".join([t["type"]["name"] for t in data["types"]])
            height = data["height"]
            weight = data["weight"]
            base_xp = data["base_experience"]
            abilities = ", ".join([a["ability"]["name"] for a in data["abilities"]])

            # Append data to CSV
            writer.writerow([pokemon_id, name, types, height, weight, base_xp, abilities])
            print(f"✅ Retrieved data for {name} (ID {pokemon_id})")

        else:
            print(f"❌ Failed to fetch Pokémon ID {pokemon_id} (Error {response.status_code})")

# Update progress file for next batch
with open(PROGRESS_FILE, "w") as f:
    f.write(str(end_index + 1))

print(f"📄 Data appended to '{CSV_FILENAME}' successfully! Next batch starts from Pokémon {end_index + 1}.")

# Upload CSV to BigQuery
def upload_to_bigquery():
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

    try:
        # Read CSV into Pandas DataFrame
        df = pd.read_csv(CSV_FILENAME)

        # Remove duplicates (if any) before uploading
        df.drop_duplicates(subset=["ID"], inplace=True)

        # Define BigQuery Schema
        job_config = bigquery.LoadJobConfig(
            schema=[
                bigquery.SchemaField("ID", "INTEGER"),
                bigquery.SchemaField("Name", "STRING"),
                bigquery.SchemaField("Type", "STRING"),
                bigquery.SchemaField("Height", "INTEGER"),
                bigquery.SchemaField("Weight", "INTEGER"),
                bigquery.SchemaField("Base XP", "INTEGER"),
                bigquery.SchemaField("Abilities", "STRING"),
            ],
            write_disposition="WRITE_TRUNCATE",  # Append new data
        )

        # Load DataFrame into BigQuery
        job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
        job.result()  # Wait for the job to complete

        print(f"🚀 Data successfully uploaded to BigQuery table {table_id}!")

    except Exception as e:
        print(f"❌ Failed to upload to BigQuery: {e}")

# Run upload function
upload_to_bigquery()


✅ Authentication successful!
✅ Retrieved data for Kabutops (ID 141)
✅ Retrieved data for Aerodactyl (ID 142)
✅ Retrieved data for Snorlax (ID 143)
✅ Retrieved data for Articuno (ID 144)
✅ Retrieved data for Zapdos (ID 145)
✅ Retrieved data for Moltres (ID 146)
✅ Retrieved data for Dratini (ID 147)
✅ Retrieved data for Dragonair (ID 148)
✅ Retrieved data for Dragonite (ID 149)
✅ Retrieved data for Mewtwo (ID 150)
📄 Data appended to 'pokemon_data.csv' successfully! Next batch starts from Pokémon 151.


f:\Python installation\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


🚀 Data successfully uploaded to BigQuery table pokemons-455012.Pokemon_Master.Master_Data!
